In [4]:
# system packages
import sys
import time
import warnings
import os

# non-geo numeric packages
import numpy as np
import math
from itertools import product, combinations
import pandas as pd

# network and OSM packages
import networkx as nx
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

# Earth engine packages
import ee
import geemap

# General geo-packages
import libpysal
import rasterio
import geopandas as gpd
import shapely
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon, MultiPolygon

In [5]:
# Authenticate and Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VNhidX5cZ-qMKUGvd6fFOgvwZRdvOFZuHqAwbrW4xNj4F2mTqfWZFk

Successfully saved authorization token.


In [18]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Cambridge'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Cambridge')

E2SCFA_score['score summary']

['United Kingdom']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\GBR_Cambridge_2020.tif
get road networks from OSM
Cambridge done 0.45 mns
 
get urban greenspaces from OSM
Cambridge done
 
100m resolution grids extraction
Cambridge 0.18 mns

get fake UGS entry points
Cambridge 0.0 % done 0.0  mns
Cambridge 48.3 % done 0.26  mns
Cambridge 96.6 % done 0.49  mns
Cambridge 100 % done 0.55  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
18.28 % 0.11 mns
36.56 % 0.27 mns
54.84 % 0.47 mns
73.13 % 0.71 mns
91.41 % 0.99 mns
100 % finding combinations done
Cambridge 1156686 suitible combinations

obtain local graphs
Cambridge
0.0 % done 0.63 mns
18.28 % done 0.71 mns
36.56 % done 0.89 mns
54.84 % done 1.18 mns
73.13 % done 1.54 mns
91.41 % done 1.99 mns
100 % done 2.25 mns

Cambridge
1.88 % done 0.3 mns
3.76 % done 0.55 mns
5.63 % done 0.83 mns
7.51 % done 1.15 mns
9.39 % done 1.43 mns
11.27 % done 1.71 mns
13.14 % done 2.07 mns
15.02 % done 2.43 mns
16.

City,Cambridge
0,"137,073.00"
Sc-access 300,231.98
M-dist 300,65.14
M-area 300,"53,626.32"
M-supply 300,85.50
Sc-norm 300,147.14
Sc-access 600,227.26
M-dist 600,240.29
M-area 600,"86,428.86"
M-supply 600,99.74


In [ ]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Liverpool'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Liverpool')

E2SCFA_score['score summary']

['United Kingdom']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\GBR_Liverpool_2020.tif
get road networks from OSM
Liverpool done 0.94 mns
 
get urban greenspaces from OSM
Liverpool done
 
100m resolution grids extraction
Liverpool 0.58 mns

get fake UGS entry points
Liverpool 0.0 % done 0.0  mns
Liverpool 38.5 % done 0.53  mns
Liverpool 76.9 % done 1.03  mns
Liverpool 100 % done 1.32  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
16.74 % 0.19 mns
33.48 % 0.41 mns
50.22 % 0.69 mns
66.96 % 0.99 mns
83.7 % 1.33 mns
100 % finding combinations done
Liverpool 1401443 suitible combinations

obtain local graphs
Liverpool
0.0 % done 1.38 mns
16.74 % done 1.46 mns
33.48 % done 1.6 mns
50.22 % done 1.8 mns
66.96 % done 2.07 mns
83.7 % done 2.39 mns
100 % done 2.75 mns

Liverpool
1.85 % done 0.42 mns
3.69 % done 0.83 mns
5.54 % done 1.2 mns
7.39 % done 1.57 mns
9.24 % done 1.99 mns
11.08 % done 2.39 mns
12.93 % done 2.88 mns
14.78 % done 3.4 mns
16.62 % 

In [ ]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Oslo'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Oslo')

E2SCFA_score['score summary']

In [21]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Chandigarh'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Chandigarh')

E2SCFA_score['score summary']

['India']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\IND_Chandigarh_2020.tif
get road networks from OSM
Chandigarh done 0.64 mns
 
get urban greenspaces from OSM
Chandigarh done
 
100m resolution grids extraction
Chandigarh 0.59 mns

get fake UGS entry points
Chandigarh 0.0 % done 0.0  mns
Chandigarh 14.1 % done 0.36  mns
Chandigarh 28.2 % done 0.73  mns
Chandigarh 42.3 % done 1.09  mns
Chandigarh 56.4 % done 1.45  mns
Chandigarh 70.5 % done 1.81  mns
Chandigarh 84.6 % done 2.19  mns
Chandigarh 98.7 % done 2.55  mns
Chandigarh 100 % done 2.61  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
16.09 % 0.17 mns
32.18 % 0.37 mns
48.26 % 0.56 mns
64.35 % 0.76 mns
80.44 % 0.97 mns
96.53 % 1.19 mns
100 % finding combinations done
Chandigarh 345485 suitible combinations

obtain local graphs
Chandigarh
0.0 % done 0.92 mns
16.09 % done 1.02 mns
32.18 % done 1.1 mns
48.26 % done 1.18 mns
64.35 % done 1.27 mns
80.44 % done 1.37 mns
96.53 % done 1.48 mns
1

City,Chandigarh
0,"893,833.00"
Sc-access 300,54.62
M-dist 300,36.33
M-area 300,"497,217.88"
M-supply 300,36.11
Sc-norm 300,1.90
Sc-access 600,52.95
M-dist 600,142.07
M-area 600,"651,607.44"
M-supply 600,37.46


In [6]:
def gee_worldpop_extract (city_file, iso, save_path = None):
    
    cities = city_file
    
    # Get included city areas
    OSM_incl = [cities[cities['City'] == city]['OSM_area'].tolist()[0].rsplit(', ') for city in cities['City'].tolist()]

    # Get the city geoms
    obj = [city_geo(city).dissolve()['geometry'].tolist()[0] for city in OSM_incl]

    # Get the city countries
    obj_displ = [city_geo(city).dissolve()['display_name'].tolist()[0].rsplit(', ')[-1]for city in OSM_incl]
    print(obj_displ)
    obj_displ = np.where(pd.Series(obj_displ).str.contains("Ivoire"),"CIte dIvoire",obj_displ)

    # Get the country's iso-code
    iso_list = [iso[iso['name'] == ob]['alpha3'].tolist()[0] for ob in obj_displ]

    # Based on the iso-code return the worldpop 2020
    ee_worldpop = [ee.ImageCollection("WorldPop/GP/100m/pop")\
        .filter(ee.Filter.date('2020'))\
        .filter(ee.Filter.inList('country', [io])).first() for io in iso_list]

    # Clip the countries with the city geoms.
    clipped = [ee_worldpop[i].clip(shapely.geometry.mapping(obj[i])) for i in range(0,len(obj))]

    # Create path if non-existent
    if save_path == None:
        path = ''
    else:
        path = save_path
        if not os.path.exists(path):
                    os.makedirs(path)

    # Export as TIFF file.
    # Stored in form path + USA_Los Angeles_2020.tif
    filenames = [path+iso_list[i]+'_'+cities['City'][i]+'_2020.tif' for i in range(len(obj))]
    [geemap.ee_export_image(clipped[i], filename = filenames[i]) for i in range(0,len(obj))]
    return(filenames)
    sys.stdout.flush()
    
    # Block 2 Road networks
def road_network (cities, thresholds, undirected = False):
    print('get road networks from OSM')
    start_time = time.time()
    graphs = list()
    road_nodes = list()
    road_edges = list()
    road_conn = list()

    for i in enumerate(cities['OSM_area']):
        # Get graph, road nodes and edges
        road_node = pd.DataFrame()
        roads = pd.DataFrame()
        
        # For each included OSM_area get the roads
        for district in i[1].rsplit(', '):
            graph = ox.graph_from_place(district, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
            node, edge = ox.graph_to_gdfs(graph)
            road_node = pd.concat([road_node, node], axis = 0)
            roads = pd.concat([roads, edge], axis = 0)
        
        # Eliminate lists in the df which prevents drop of duplicate columns
        road_edge = pd.DataFrame([[c[0] if isinstance(c,list) else c for c in roads[col]]\
                              for col in roads]).transpose()
        road_edge.columns = roads.columns
        road_edge.index = roads.index
        road_edge = gpd.GeoDataFrame(road_edge, crs = 4326)
        
        # Return the unique nodes and edges of the (often) adjacent OSM_areas.
        road_node = road_node.drop_duplicates()
        road_edge = road_edge.drop_duplicates()
        
        # Road nodes format
        road_node = road_node.to_crs(4326)
        road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
        road_node['osmid_var'] = road_node.index
        road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)

        # format road edges
        road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
        road_edge = road_edge.reset_index()
        road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
        road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
        
        if undirected == True:
            # Apply one-directional to both for walking
            both = road_edge[road_edge['oneway'] == False]
            one = road_edge[road_edge['oneway'] == True]
            rev = pd.DataFrame()
            rev[['from','to']] = one[['to','from']]
            rev = pd.concat([rev,one.iloc[:,2:]],axis = 1)
            edge_bidir = pd.concat([both, one, rev])
            edge_bidir = edge_bidir.reset_index()
            edge_bidir['oneway'] = False
        else:
            edge_bidir = road_edge

        # Exclude highways and ramps on edges    
        edge_filter = edge_bidir[(edge_bidir['highway'].str.contains('motorway') | 
              (edge_bidir['highway'].str.contains('trunk') & 
               edge_bidir['maxspeed'].astype(str).str.contains(
                   '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
        road_edges.append(edge_filter)

        # Exclude isolated nodes
        fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
        newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
        road_nodes.append(newnodes)

        # Get only necessary road connections columns for network performance
        road_con = edge_filter[['osmid','key','length','geometry']]
        road_con = road_con.set_index('key')

        road_conn.append(road_con)

        # formatting to graph again.
        newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
        edge_filter = edge_filter.set_index(['from','to','keys'])
        edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]

        graph2 = ox.graph_from_gdfs(newnodes, edge_filter)

        graphs.append(graph2)
        print(cities['City'][i[0]].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    return({'graphs':graphs,'nodes':road_nodes,'edges':road_conn,'edges long':road_edges})
# Block 3 city greenspace
def urban_greenspace (cities, thresholds, one_UGS_buf = 25, min_UGS_size = 400):
    print('get urban greenspaces from OSM')
    parks_in_range = list()
    for i in enumerate(cities['OSM_area']):
        # Tags seen as Urban Greenspace (UGS) require the following:
        # 1. Tag represent an area
        # 2. The area is outdoor
        # 3. The area is (semi-)publically available
        # 4. The area is likely to contain trees, grass and/or greenery
        # 5. The area can reasonable be used for walking or recreational activities
        tags = {'landuse':['allotments','forest','greenfield','village_green'],\
                'leisure':['garden','fitness_station','nature_reserve','park','playground'],\
                'natural':'grassland'}
        gdf = ox.geometries_from_place(i[1].rsplit(', '),tags = tags,buffer_dist = np.max(thresholds))
        gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
        greenspace = gdf.reset_index()    
        warnings.filterwarnings("ignore")

        green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_UGS_buf).to_crs(4326))
        greenspace['geometry_w_buffer'] = green_buffer
        greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
        greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

        # This function group components in itself that overlap (with the buffer set of 25 metres)
        # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
        W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
        greenspace['components'] = W.component_labels
        parks = greenspace.dissolve('components')

        # Exclude parks below 0.04 ha.
        parks = parks[parks.to_crs(3043).area > min_UGS_size]
        print(cities['City'][i[0]], 'done')
        parks = parks.reset_index()
        parks['geometry_m'] = parks['geometry'].to_crs(3043)
        parks['park_area'] = parks['geometry_m'].area
        parks_in_range.append(parks)
    return(parks_in_range)
# Block 4 population grids extraction
def city_grids_format(city_grids, cities_area, road_nodes, UGS, grid_size = 100):
    start_time = time.time()
    grids = []
    print(str(grid_size) + 'm resolution grids extraction')
    for i in range(len(city_grids)):
        
        # Open the raster file
        with rasterio.open(city_grids[i]) as src:
            band= src.read() # the population values
            aff = src.transform # the raster bounds and size (affine)
        
        # Get the rowwise arrays, get a 2D dataframe
        grid = pd.DataFrame()
        for b in enumerate(band[0]):
            grid = pd.concat([grid, pd.Series(b[1],name=b[0])],axis=1)
        grid= grid.unstack().reset_index()
        
        # Unstack df to columns
        grid.columns = ['row','col','value']
        grid['minx'] = aff[2]+aff[0]*grid['col']
        grid['miny'] = aff[5]+aff[4]*grid['row']
        grid['maxx'] = aff[2]+aff[0]*grid['col']+aff[0]
        grid['maxy'] = aff[5]+aff[4]*grid['row']+aff[4]
        
        # Create polygon from affine bounds and row/col indices
        grid['geometry'] = [Polygon([(grid.minx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.maxy[i]),
                                   (grid.minx[i],grid.maxy[i])])\
                          for i in range(len(grid))]
        
        # Set the df as geo-df
        grid = gpd.GeoDataFrame(grid, crs = 4326) 

        # Get dissolvement_key for dissolvement. 
        grid['row3'] = np.floor(grid['row']/(grid_size/100)).astype(int)
        grid['col3'] = np.floor(grid['col']/(grid_size/100)).astype(int)
        grid['dissolve_key'] = grid['row3'].astype(str) +'-'+ grid['col3'].astype(str)
        
        # Define a city's OSM area as Polygon.
        geo_ls = gpd.GeoSeries(city_geo(cities_area[i].split(', ')).dissolve().geometry)
        
        # Intersect grids with the city boundary Polygon.
        insec = grid.intersection(geo_ls.tolist()[0])
        
        # Exclude grids outside the specified city boundaries
        insec = insec[insec.area > 0]
        
        # Join in other information.
        insec = gpd.GeoDataFrame(geometry = insec, crs = 4326).join(grid.loc[:, grid.columns != 'geometry'])
        
        # Dissolve into block by block grids
        popgrid = insec[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')
        
        # Get those grids populations and area. Only blocks with population and full blocks
        popgrid['population'] = round(insec.groupby('dissolve_key')['value'].sum()).astype(int)
        popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
        popgrid = popgrid[popgrid['population'] > 0]
        popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]

        # Get centroids and coords
        popgrid['centroid'] = popgrid['geometry'].centroid
        popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
        popgrid['grid_lon'] = popgrid['centroid_m'].x
        popgrid['grid_lat'] = popgrid['centroid_m'].y
        popgrid = popgrid.reset_index()

        minx = popgrid.bounds['minx']
        maxx = popgrid.bounds['maxx']
        miny = popgrid.bounds['miny']
        maxy = popgrid.bounds['maxy']

        # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters), coords error.
        # Therefore recreate the polygon.
        Poly = []
        for k in range(len(popgrid)):
            Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
        popgrid['geometry'] = Poly
        
        try:
            entry_index = [int(road_nodes[i]['geometry'].sindex.nearest(grid)[1])\
                                 for grid in popgrid['centroid']]
        except:
            entry_index = [int(road_nodes[i]['geometry'].sindex.nearest(grid)[1][0])\
                                 for grid in popgrid['centroid']]
            
        nearest_index = road_nodes[i].iloc[entry_index]
        popgrid['grid_osm'] = nearest_index.reset_index(drop = True)['osmid_var']
        popgrid['node_geom'] = nearest_index.reset_index(drop = True)['geometry']
        popgrid['node_geom_m'] = nearest_index.reset_index(drop = True)['geometry_m']
        popgrid['G-entry cost'] = popgrid['node_geom_m'].distance(popgrid['centroid_m'])
        
        UGS_all = UGS[i].dissolve().geometry[0]
        popgrid['in_out_UGS'] = popgrid.intersection(UGS_all).is_empty == False
        
        grids.append(popgrid)

        print(city_grids[i].rsplit('_')[3], round((time.time() - start_time)/60,2),'mns')
    return(grids)

In [7]:
# Block 5 park entry points
def UGS_fake_entry(UGS, road_nodes, graphs, cities, pop_grids,
                   thresholds, UGS_entry_buf = 25, walk_radius = 500, entry_point_merge = 0):
    print('get fake UGS entry points')
    start_time = time.time()
    ParkRoads = list()
    for j in range(len(cities)):
        ParkRoad = pd.DataFrame()
        mat = list()
        # For all
        for i in range(len(UGS[j])):
            dist = road_nodes[j]['geometry'].to_crs(3043).distance(UGS[j]['geometry'].to_crs(
                3043)[i])
            buf_nodes = road_nodes[j][(dist < UGS_entry_buf) & (dist > 0)]
            mat.append(list(np.repeat(i, len(buf_nodes))))
            ParkRoad = pd.concat([ParkRoad, buf_nodes])
            if i % 100 == 0: print(cities[j].rsplit(',')[0], round(i/len(UGS[j])*100,1),'% done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        # Park no list conversion
        mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

        # Format
        ParkRoad['Park_No'] = mat_u
        ParkRoad = ParkRoad.reset_index()
        ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
        ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
        
        # Get the road nodes intersecting with the parks' buffer
        ParkRoad = pd.merge(ParkRoad, UGS[j][['geometry','park_area']], left_on = 'Park_No', right_index = True)

        # Get the walkable park size
        ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walk_radius).to_crs(4326).intersection(ParkRoad['geometry_y'].to_crs(4326))
        ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
        ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
        ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
                
        # Merge fake UGS entry points if within X meters of each other for better system performance
        # Standard no merging
        ParkRoad = simplify_UGS_entry(ParkRoad, entry_point_merge = 0)
                
        ParkRoads.append(ParkRoad)

        print(cities[j].rsplit(',')[0],'100 % done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        
    return(ParkRoads)
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)
def simplify_UGS_entry(fake_UGS_entry, entry_point_merge = 0):
    # Get buffer of nodes close to each other.
    # Get the buffer
    ParkComb = fake_UGS_entry
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(entry_point_merge)

    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels

    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']

    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    return(ParkComb)

In [8]:
def suitible_enhanced (UGS_entry, pop_grids, road_nodes, cities, thresholds):
    start_time = time.time()
    suits_all = []
    for j in range(len(cities)):
        print('get (Euclidean) suitible combinations')
        print('0.0 %', round((time.time() - start_time) / 60,2),'mns')
        UGSe = UGS_entry[j]
        entry_geoms = UGSe.geometry_m
        pop = pop_grids[j]
        road_node = road_nodes[j]

        suits = pd.DataFrame()
        cols = ['osmid','Park_No','park_area']
        for i in range(len(entry_geoms)):
            suit_df = pop[pop.node_geom_m.distance(entry_geoms.iloc[i]) < np.max(thresholds)]
        
            suit_df['UGSe_osmid_m'] = entry_geoms.iloc[i]
            suit_df['Grid_No'] = suit_df.index
            suit_df = suit_df[['Grid_No','grid_osm','G-entry cost','in_out_UGS','node_geom_m','UGSe_osmid_m']].reset_index(drop = True)
            suit_df['Park_entry_No'] = UGSe.index[i]
            suits = pd.concat([suits,suit_df])
            if (i+1) % 500 == 0: print(round((i+1) / len(entry_geoms)*100,2),'%',
                                       round((time.time() - start_time) / 60,2),'mns')
            
        suits = pd.merge(suits, UGSe[cols], left_on = 'Park_entry_No',right_index = True, how = 'left')
        suits = suits.reset_index(drop = True)
        suits = suits.rename(columns = {'osmid':'Parkroad_osmid','park_area':'park_area_m2'})
        suits['gridpark_no'] = suits['Grid_No'].astype(str)+'-'+suits['Park_No'].astype(str)
        suits['graph_key'] = suits['grid_osm'].astype(str)+'-'+suits['Parkroad_osmid'].astype(str)
        suits_all.append(suits)
        print('100 % finding combinations done')
        print(cities[j],len(suits),'suitible combinations')
    return(suits_all)

In [9]:
def obtaining_subgraphs(graphs, pop_grids, UGS_entry, nodes, cities, thresholds):
    print('obtain local graphs')
    start_time = time.time()
    subgraphs_all = []
    suits_all = []
    for j in range(len(cities)):
        print(cities[j])
        Graph = graphs[j]
        pop = pop_grids[j]
        UGSe = UGS_entry[j].sort_values('osmid')
        road_node = nodes[j]
        node_geoms = road_node.geometry_m
        entry_geoms = UGSe.geometry_m
        osmid = UGSe['osmid']

        dist = [node_geoms.distance(Point(i)) for i in entry_geoms]

        print('0.0 % done',round((time.time() - start_time) / 60,2),'mns')
        subgraphs = []
        UGSe_ids = []
        suits = pd.DataFrame()
        for i in range(len(entry_geoms)):      
            suit = road_node[['geometry_m']]
            suit['UGSe_osmid_m'] = entry_geoms.iloc[i]
            suit_df = dist[i]
            suit_in = suit_df[suit_df <= np.max(thresholds)]
            UGSe_ids.append(osmid.iloc[i])
            suit_in = pd.DataFrame(suit_in).join(node_geoms)
            suit_in['Parkroad_osmid'] = osmid.iloc[i]
            subgraphs.append(Graph.subgraph(suit_in.index))
            suits = pd.concat([suits, suit_in])

            if (i+1) % 500 == 0: print(round((i+1) / len(entry_geoms)*100,2),'% done',
                                        round((time.time() - start_time) / 60,2),'mns')
        print('100 % done',round((time.time() - start_time) / 60,2),'mns')
        subgraphs_all.append(pd.Series(subgraphs, index = UGSe_ids))
        suits_all.append(suits)
    return({'graphs':subgraphs_all,'graph nodes':suits_all})

In [10]:
def distance_fast (Geo_1, Geo_2):
    return((abs(Geo_1.x - Geo_2.x)**2 + abs(Geo_1.y - Geo_2.y)**2).apply(math.sqrt))

In [11]:
def direct_routing (suitible_comb, graphs, edges, cities):
    start_time = time.time()
    Routes = []
    for j in enumerate(cities):
        print(j[1])
        comb = suitible_comb[j[0]]
        grouped = comb[comb['in_out_UGS'] == False].groupby(['Parkroad_osmid'])['grid_osm'].apply(list)
        sets = grouped.apply(np.unique)
        
        parknode = list(comb['Parkroad_osmid'])
        gridnode = list(comb['grid_osm'])
        Conn = edges[j[0]]
        subgraph = graphs[j[0]]
        subgraph = subgraph[sets.index]

        ls = []
        ls2 = []
        ls3 = []
        linestr = pd.Series()
        for i in range(len(sets)):
            path = nx.single_source_dijkstra(subgraph.iloc[i], sets.index[i], weight = 'length')

            # Only include routes that were prespecified, order depends on route cost, low to high, and steps low to high.
            # Grid destinations from UGS entry points therefore may be ranked differently, and subsetted separately.
            incl = np.isin(list(path[0].keys()),sets.iloc[i])
            incl2 = np.isin(list(path[1].keys()),sets.iloc[i])

            # route cost
            orig_c = list(np.repeat(sets.index[i],sum(incl)))
            dest_c = list(np.array(list(path[0].keys()))[incl])
            cost = list(np.array(list(path[0].values()))[incl])

            ls = ls + orig_c
            ls2= ls2+ dest_c
            ls3= ls3+ cost

            # route steps
            orig_s = list(np.repeat(sets.index[i],sum(incl2)))
            dest_s = list(np.array(list(path[1].keys()))[incl2])
            steps = list(np.array(list(path[1].values()))[incl2])

            fr = []
            to = []
            og = []
            de = []
            for j in enumerate(steps):
                fr.append(j[1][:-1])
                to.append(j[1][1:])
                og.append(list(np.repeat(orig_s[j[0]], len(j[1][:-1]))))
                de.append(list(np.repeat(dest_s[j[0]], len(j[1][:-1]))))
                
            fr = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, fr) for i in b]
            to = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, to) for i in b]
            og = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, og) for i in b]
            de = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, de) for i in b]
            
            gk = [str(fr[k])+'-'+str(to[k]) for k in range(len(to))]
            gkr = [str(to[k])+'-'+str(fr[k]) for k in range(len(to))]
            od = [str(de[k])+'-'+str(og[k]) for k in range(len(og))]

            if len(od) > 0:
                ser_gk = pd.DataFrame(gkr, index = gk)
                ser_gk['od'] = od
                ser_gk = ser_gk.join(Conn.geometry, how = 'left')
                ser_gk = ser_gk.set_index(ser_gk.columns[0], drop = True)
                ser_gk = ser_gk.join(Conn.geometry, how = 'left', rsuffix = '_r')
                ser_gk['geom'] = np.where(ser_gk.geometry.isna(),ser_gk.geometry_r,ser_gk.geometry)
                diss = gpd.GeoDataFrame(ser_gk[['od','geom']], geometry = 'geom', crs = 4326).dissolve(by = 'od')
                diss = diss.geom
                linestr = pd.concat([linestr, diss], axis = 0)

            if (i+1) % 50 == 0: print(round((i+1) / len(sets)*100,2),'% done',round((time.time() - start_time) / 60,2),'mns')

        linestr = linestr.rename('geometry')

        dist_df = pd.DataFrame([ls, ls2, ls3]).transpose()
        dist_df.columns = ['UGSe_id','GrE_id','route cost']
        dist_df['UGSe_id'] = [int(i) for i in dist_df['UGSe_id']]
        dist_df['GrE_id'] = [int(i) for i in dist_df['GrE_id']]
        dist_df['graph_key'] = dist_df['GrE_id'].astype(str)+'-'+dist_df['UGSe_id'].astype(str)

        routes = pd.merge(comb, dist_df, on = 'graph_key', how = 'left')
        routes['route cost'] = np.where(routes['in_out_UGS'],0,routes['route cost'])
        routes['G-entry cost'] = np.where(routes['in_out_UGS'],0,routes['G-entry cost'])
        
        routes['Tcost'] = routes['route cost']+routes['G-entry cost']
        
        routes = pd.merge(routes, linestr, left_on = 'graph_key', right_index = True, how = 'left')
        
        routes2 = routes.iloc[routes['route cost'].dropna().index].reset_index(drop = True)
        
        print('100 % done',round((time.time() - start_time) / 60,2),'mns')
        
        Routes.append(routes2)
    return(Routes)

In [12]:
def min_gridUGS_comb (routes, grids, UGS):
    gp_nearest = []
    for i in range(len(routes)):
        gp_nn = routes[i][routes[i]['Tcost'] <= max(thresholds)]
        gp_nn = pd.merge(gp_nn, grids[i]['population'], left_on='Grid_No', right_index = True)
        gp_nn = pd.merge(gp_nn, UGS[i]['park_area'], left_on = 'Park_No', right_index = True)
        gp_nn = gp_nn.reset_index()

        gp_nn = gp_nn.iloc[gp_nn.groupby('gridpark_no')['Tcost'].idxmin()]
        gp_nn.index.name = 'idx'
        gp_nn = gp_nn.sort_values('idx')
        gp_nn = gp_nn.reset_index()
        gp_nearest.append(gp_nn)
    gp_nearest[0].sort_values('Grid_No')
    return(gp_nearest)

In [13]:
def E2SCFA_scores(min_gridUGS_comb, grids, thresholds, cities, 
                  save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA/', grid_size = 100, ext = ''):
    pd.options.display.float_format = '{:20,.2f}'.format
    E2SFCA_cities = []
    E2SFCA_summary = pd.DataFrame()
    for i in range(len(cities)):
        E2SFCA_score = grids[i][['population','geometry']]
        for j in range(len(thresholds)):
            subset = min_gridUGS_comb[i][min_gridUGS_comb[i]['Tcost'] <= thresholds[j]]

            # use gussian distribution: let v= 923325, then the weight for 800m is 0.5
            v = -thresholds[j]**2/np.log(0.5)

            # add a column of weight: apply the decay function on distance
            subset['weight'] = np.exp(-(subset['Tcost']**2/v)).astype(float)
            subset['pop_weight'] = subset['weight'] * subset['population']

            # get the sum of weighted population each green space has to serve.
            s_w_p = pd.DataFrame(subset.groupby('Park_No').sum('pop_weight')['pop_weight'])

            # delete other columns, because they are useless after groupby
            s_w_p = s_w_p.rename({'pop_weight':'pop_weight_sum'},axis = 1)
            middle = pd.merge(subset,s_w_p, how = 'left', on = 'Park_No' )

            # calculate the supply-demand ratio for each green space
            middle['green_supply'] = middle['park_area']/middle['pop_weight_sum']

            # caculate the accessbility score for each green space that each population grid cell could reach
            middle['Sc-access'] = middle['weight'] * middle['green_supply']
            # add the scores for each population grid cell
            pop_score_df = pd.DataFrame(middle.groupby('Grid_No').sum('Sc-access')['Sc-access'])

            # calculate the mean distance of all the green space each population grid cell could reach
            mean_dist = middle.groupby('Grid_No').mean('Tcost')['Tcost']
            pop_score_df['M-dist'] = mean_dist

            # calculate the mean area of all the green space each population grid cell could reach
            mean_area = middle.groupby('Grid_No').mean('park_area')['park_area']
            pop_score_df['M-area'] = mean_area

            # calculate the mean supply_demand ratio of all the green space each population grid cell could reach
            mean_supply = middle.groupby('Grid_No').mean('green_supply')['green_supply']
            pop_score_df['M-supply'] = mean_supply

            pop_score = pop_score_df

            pop_score_df = pop_score_df.join(grids[i]['population'], how = 'right')
            pop_score_df['Sc-norm'] = pop_score_df['Sc-access'] / pop_score_df['population']

            pop_score_df = pop_score_df.loc[:, pop_score_df.columns != 'population']
            pop_score_df = pop_score_df.add_suffix(' '+str(thresholds[j]))
            E2SFCA_score = E2SFCA_score.join(pop_score_df, how = 'left')

            print(thresholds[j], cities[i])

        E2SFCA_score = E2SFCA_score.fillna(0)
        
        if not os.path.exists(save_path+str(grid_size)+'m grids'+'/grid_geoms/'):
            os.makedirs(save_path+str(grid_size)+'m grids'+'/grid_geoms/')
        
        E2SFCA_score.to_file(save_path+str(grid_size)+'m grids'+'/grid_geoms/'+cities[i]+'.gpkg') # Detailed scores
        pop_sum = pd.Series(E2SFCA_score['population'].sum()).astype(int)
        mean_metrics = E2SFCA_score.loc[:, ~E2SFCA_score.columns.isin(['population','geometry'])].mean()
        E2SFCA_sum = pd.concat([pop_sum, mean_metrics])
        E2SFCA_summary = pd.concat([E2SFCA_summary, E2SFCA_sum], axis = 1) # summarized results
        E2SFCA_cities.append(E2SFCA_score)
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        E2SFCA_score.loc[:, E2SFCA_score.columns != 'geometry'].to_csv(save_path+cities[i]+'.csv')
    E2SFCA_summary.columns = cities
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    E2SFCA_summary.to_csv(save_path+str(grid_size)+'m grids'+'all_cities'+ext+'.csv')
    E2SFCA_summary
    return({'score summary':E2SFCA_summary,'score detail':E2SFCA_cities})